In [1]:
import keras
import numpy as np
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, AveragePooling2D
from keras.initializers import RandomNormal  
from keras import optimizers
from keras.callbacks import LearningRateScheduler, TensorBoard
from keras.layers.normalization import BatchNormalization

batch_size    = 128
epochs        = 200
iterations    = 391
num_classes   = 10
dropout       = 0.5
weight_decay  = 0.0001
log_filepath  = './nin'

def color_preprocessing(x_train,x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    mean = [125.307, 122.95, 113.865]
    std  = [62.9932, 62.0887, 66.7048]
    for i in range(3):
        x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
        x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]

    return x_train, x_test

def scheduler(epoch):
    if epoch <= 80:
        return 0.01
    if epoch <= 140:
        return 0.005
    return 0.001

def build_model():
  model = Sequential()

  model.add(Conv2D(192, (5, 5), padding='same', kernel_regularizer=keras.regularizers.l2(weight_decay), kernel_initializer="he_normal", input_shape=x_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(160, (1, 1), padding='same', kernel_regularizer=keras.regularizers.l2(weight_decay), kernel_initializer="he_normal"))
  model.add(Activation('relu'))
  model.add(Conv2D(96, (1, 1), padding='same', kernel_regularizer=keras.regularizers.l2(weight_decay), kernel_initializer="he_normal"))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(3, 3),strides=(2,2),padding = 'same'))
  
  model.add(Dropout(dropout))
  
  model.add(Conv2D(192, (5, 5), padding='same', kernel_regularizer=keras.regularizers.l2(weight_decay), kernel_initializer="he_normal"))
  model.add(Activation('relu'))
  model.add(Conv2D(192, (1, 1),padding='same', kernel_regularizer=keras.regularizers.l2(weight_decay), kernel_initializer="he_normal"))
  model.add(Activation('relu'))
  model.add(Conv2D(192, (1, 1),padding='same', kernel_regularizer=keras.regularizers.l2(weight_decay), kernel_initializer="he_normal"))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(3, 3),strides=(2,2),padding = 'same'))
  
  model.add(Dropout(dropout))
  
  model.add(Conv2D(192, (3, 3), padding='same', kernel_regularizer=keras.regularizers.l2(weight_decay), kernel_initializer="he_normal"))
  model.add(Activation('relu'))
  model.add(Conv2D(192, (1, 1), padding='same', kernel_regularizer=keras.regularizers.l2(weight_decay), kernel_initializer="he_normal"))
  model.add(Activation('relu'))
  model.add(Conv2D(10, (1, 1), padding='same', kernel_regularizer=keras.regularizers.l2(weight_decay), kernel_initializer="he_normal"))
  model.add(Activation('relu'))
  
  model.add(GlobalAveragePooling2D())
  model.add(Activation('softmax'))
  
  sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
  model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

if __name__ == '__main__':

    # load data
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    
    x_train, x_test = color_preprocessing(x_train, x_test)

    # build network
    model = build_model()
    print(model.summary())

    # set callback
    tb_cb = TensorBoard(log_dir=log_filepath, histogram_freq=0)
    change_lr = LearningRateScheduler(scheduler)
    cbks = [change_lr,tb_cb]

    # set data augmentation
    print('Using real-time data augmentation.')
    datagen = ImageDataGenerator(horizontal_flip=True,width_shift_range=0.125,height_shift_range=0.125,fill_mode='constant',cval=0.)
    datagen.fit(x_train)

    # start training
    model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),steps_per_epoch=iterations,epochs=epochs,callbacks=cbks,validation_data=(x_test, y_test))
    model.save('nin.h5')


Using TensorFlow backend.


170500096/170498071 [==============================] - 108s   
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 192)       14592     
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 192)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 160)       30880     
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 160)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 96)        15456     
_________________________________________________________________
activation_3 (Activation)    (None, 32, 32, 96)        0         
_________________________________________________________________
max_pooling2d

391/391 [==============================] - 61s - loss: 1.1854 - acc: 0.6591 - val_loss: 1.1935 - val_acc: 0.6565
Epoch 37/200
391/391 [==============================] - 61s - loss: 1.1768 - acc: 0.6621 - val_loss: 1.2331 - val_acc: 0.6552
Epoch 38/200
391/391 [==============================] - 61s - loss: 1.1656 - acc: 0.6682 - val_loss: 1.1683 - val_acc: 0.6658
Epoch 39/200
391/391 [==============================] - 61s - loss: 1.1583 - acc: 0.6669 - val_loss: 1.1940 - val_acc: 0.6657
Epoch 40/200
391/391 [==============================] - 61s - loss: 1.1508 - acc: 0.6698 - val_loss: 1.2138 - val_acc: 0.6648
Epoch 41/200
391/391 [==============================] - 61s - loss: 1.1430 - acc: 0.6716 - val_loss: 1.1427 - val_acc: 0.6763
Epoch 42/200
391/391 [==============================] - 61s - loss: 1.1350 - acc: 0.6755 - val_loss: 1.1307 - val_acc: 0.6725
Epoch 43/200
391/391 [==============================] - 61s - loss: 1.1273 - acc: 0.6796 - val_loss: 1.1864 - val_acc: 0.6710
Epoch

391/391 [==============================] - 61s - loss: 0.8992 - acc: 0.7537 - val_loss: 1.0748 - val_acc: 0.7231
Epoch 102/200
391/391 [==============================] - 61s - loss: 0.8995 - acc: 0.7529 - val_loss: 1.0469 - val_acc: 0.7229
Epoch 103/200
391/391 [==============================] - 61s - loss: 0.8938 - acc: 0.7579 - val_loss: 1.0687 - val_acc: 0.7244
Epoch 104/200
391/391 [==============================] - 61s - loss: 0.8932 - acc: 0.7585 - val_loss: 1.0293 - val_acc: 0.7285
Epoch 105/200
391/391 [==============================] - 61s - loss: 0.8890 - acc: 0.7599 - val_loss: 1.0583 - val_acc: 0.7193
Epoch 106/200
391/391 [==============================] - 61s - loss: 0.8922 - acc: 0.7567 - val_loss: 1.0615 - val_acc: 0.7207
Epoch 107/200
391/391 [==============================] - 61s - loss: 0.8899 - acc: 0.7580 - val_loss: 1.0296 - val_acc: 0.7263
Epoch 108/200
391/391 [==============================] - 61s - loss: 0.8878 - acc: 0.7586 - val_loss: 1.0320 - val_acc: 0.726

391/391 [==============================] - 61s - loss: 0.8090 - acc: 0.7850 - val_loss: 1.0277 - val_acc: 0.7383
Epoch 166/200
391/391 [==============================] - 61s - loss: 0.8051 - acc: 0.7876 - val_loss: 1.0189 - val_acc: 0.7414
Epoch 167/200
391/391 [==============================] - 61s - loss: 0.8068 - acc: 0.7850 - val_loss: 1.0027 - val_acc: 0.7423
Epoch 168/200
391/391 [==============================] - 61s - loss: 0.8037 - acc: 0.7863 - val_loss: 1.0407 - val_acc: 0.7371
Epoch 169/200
391/391 [==============================] - 61s - loss: 0.8065 - acc: 0.7831 - val_loss: 1.0143 - val_acc: 0.7419
Epoch 170/200
391/391 [==============================] - 61s - loss: 0.8056 - acc: 0.7842 - val_loss: 1.0234 - val_acc: 0.7399
Epoch 171/200
391/391 [==============================] - 61s - loss: 0.8053 - acc: 0.7849 - val_loss: 1.0385 - val_acc: 0.7335
Epoch 172/200
391/391 [==============================] - 61s - loss: 0.8065 - acc: 0.7850 - val_loss: 1.0232 - val_acc: 0.739